<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BANKING_AAI_GROK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xai-sdk -q

In [5]:
import datetime
import json
import time # For simulating delay

# --- 1. Conceptual Environment ---
class BankingEnvironment:
    """Simulates the banking data environment an agent perceives."""
    def __init__(self):
        self.customer_data = {
            "customer_123": {
                "age": 35,
                "income": 75000,
                "current_products": ["checking_account", "savings_account"],
                "recent_activity": ["large_deposit", "mortgage_inquiry_web"],
                "credit_score": 780,
                "location": "urban"
            },
            "customer_456": {
                "age": 28,
                "income": 40000,
                "current_products": ["checking_account"],
                "recent_activity": ["frequent_small_transactions", "car_purchase_inquiry"],
                "credit_score": 650,
                "location": "suburban"
            },
            "customer_789": {
                "age": 55,
                "income": 120000,
                "current_products": ["checking_account", "savings_account", "mortgage"],
                "recent_activity": ["retirement_planning_inquiry"],
                "credit_score": 810,
                "location": "rural"
            }
        }
        self.product_catalog = {
            "mortgage": {"eligibility": {"min_credit_score": 700, "min_income": 60000}, "description": "Home loan for property purchase."},
            "credit_card_rewards": {"eligibility": {"min_credit_score": 720}, "description": "Credit card with rewards points."},
            "personal_loan": {"eligibility": {"min_credit_score": 600, "min_income": 30000}, "description": "Loan for personal expenses."},
            "investment_portfolio": {"eligibility": {"min_income": 80000}, "description": "Diversified investment options for wealth growth."},
            "travel_insurance": {"eligibility": {}, "description": "Insurance for travel protection."},
            "retirement_planning_service": {"eligibility": {"min_age": 50}, "description": "Expert guidance for retirement planning."},
            "student_loan_refinance": {"eligibility": {"max_age": 40}, "description": "Refinance student loans for better rates."} # Added for diversity
        }
        print("BankingEnvironment initialized.")

    def get_customer_profile(self, customer_id):
        """Retrieves simulated customer data."""
        return self.customer_data.get(customer_id)

    def get_product_details(self, product_name):
        """Retrieves simulated product details."""
        return self.product_catalog.get(product_name)

    def get_all_products(self):
        """Returns a list of all available product names."""
        return list(self.product_catalog.keys())

# --- 2. Agent Memory (Simplified) ---
class AgentMemory:
    """A simple conceptual memory for an agent."""
    def __init__(self):
        self.past_recommendations = {}
        self.knowledge_base = {
            "product_descriptions": {name: details["description"] for name, details in BankingEnvironment().product_catalog.items()}
        }
        print("AgentMemory initialized.")

    def record_recommendation(self, customer_id, product_name, timestamp):
        """Records a recommendation made to a customer."""
        if customer_id not in self.past_recommendations:
            self.past_recommendations[customer_id] = []
        self.past_recommendations[customer_id].append({"product": product_name, "timestamp": timestamp})
        print(f"Memory: Recorded recommendation for {customer_id}: {product_name}")

    def get_past_recommendations(self, customer_id):
        """Retrieves past recommendations for a customer."""
        return self.past_recommendations.get(customer_id, [])

    def get_knowledge(self, key):
        """Retrieves conceptual knowledge from memory."""
        return self.knowledge_base.get(key)

# --- 3. The Agent Itself ---
class CrossSellingAgent:
    """
    An Agentic AI component focused on identifying the Next Best Service (NBS).
    It perceives, reasons (using an LLM), and acts.
    """
    def __init__(self, agent_id, environment: BankingEnvironment, memory: AgentMemory, llm_client):
        self.agent_id = agent_id
        self.environment = environment
        self.memory = memory
        self.llm_client = llm_client # The XAI SDK client (now correctly mocked)
        self.goal = "Maximize customer value through highly relevant and personalized product recommendations."
        print(f"\nCrossSellingAgent '{self.agent_id}' initialized with goal: '{self.goal}'")

    def perceive(self, customer_id):
        """
        Perceives relevant information about a customer from the environment.
        """
        print(f"\nAgent {self.agent_id}: Perceiving data for customer {customer_id}...")
        customer_profile = self.environment.get_customer_profile(customer_id)
        if customer_profile:
            # Add customer_id to profile for easier reference in reasoning
            customer_profile["customer_id"] = customer_id
            print(f"Agent {self.agent_id}: Perceived customer profile: {customer_profile}")
            return customer_profile
        else:
            print(f"Agent {self.agent_id}: Customer {customer_id} not found.")
            return None

    def reason(self, customer_profile):
        """
        Reasons based on perceived data to determine the next best product using an LLM.
        This includes checking against past recommendations from memory.
        """
        if not customer_profile:
            return None

        customer_id = customer_profile.get("customer_id", "Unknown Customer")
        print(f"Agent {self.agent_id}: Reasoning for customer {customer_id} using LLM...")

        # Prepare context for the LLM
        all_products = self.environment.get_all_products()
        product_details = {p: self.environment.get_product_details(p) for p in all_products}
        past_recommendations = self.memory.get_past_recommendations(customer_id)
        past_rec_product_names = {rec["product"].lower() for rec in past_recommendations}
        current_owned_products = {p.lower() for p in customer_profile.get("current_products", [])}

        # Construct the prompt for the LLM
        prompt_text = f"""
        You are an AI assistant for a banking agent. Your goal is to recommend the single most relevant "Next Best Service" (NBS) for a customer based on their profile and recent activities.
        Consider the customer's current products, recent activities, credit score, income, and age.
        It is CRITICAL that you do NOT recommend products the customer already has, or products that have been recently recommended to them.

        Customer Profile:
        {json.dumps(customer_profile, indent=2)}

        Available Products and their Eligibility (simplified):
        {json.dumps(product_details, indent=2)}

        Customer's Past Recommendations:
        {json.dumps(past_recommendations, indent=2)}

        Customer's Current Products (do not recommend these):
        {json.dumps(list(current_owned_products), indent=2)}

        Products ALREADY RECOMMENDED (do not recommend these again):
        {json.dumps(list(past_rec_product_names), indent=2)}

        Your task is to identify ONE product that is most suitable and that the customer does NOT currently have and has NOT been recently recommended.
        Output ONLY the name of the recommended product. If no suitable product is found, output "None".
        """

        try:
            # Get the chat session object from the mocked client
            chat_session = self.llm_client.chat.create(model="grok-4-0709", temperature=0)

            # Append system and user messages to the chat history
            # Using the mock user/system message classes if xai_sdk is not truly imported
            chat_session.append(system("You are a helpful banking product recommendation AI. You respond with ONLY the product name or 'None'."))
            chat_session.append(user(prompt_text))

            # --- START SIMULATED LLM RESPONSE ---
            print(f"Agent {self.agent_id}: (Simulating LLM call for customer {customer_id})")
            # These mock responses are now chosen based on a more complex logic to show the memory effect
            # For customer_123, if 'mortgage' is already recommended, we would expect a different mock here
            # For demonstration, let's hardcode specific sequences for `customer_123`
            if customer_id == "customer_123":
                if "mortgage" in past_rec_product_names:
                    llm_raw_response = "credit_card_rewards" # Next best if mortgage done
                else:
                    llm_raw_response = "mortgage"
            elif customer_id == "customer_456":
                llm_raw_response = "personal_loan"
            elif customer_id == "customer_789":
                llm_raw_response = "retirement_planning_service"
            else:
                llm_raw_response = "None"

            time.sleep(1) # Simulate delay
            # --- END SIMULATED LLM RESPONSE ---

            # In a real scenario, you would uncomment the line below:
            # response = chat_session.sample() # Call sample on the chat_session object
            # llm_raw_response = response.content

            print(f"Agent {self.agent_id}: LLM Raw Response: '{llm_raw_response}'")

            # Parse the LLM's response
            recommended_product = llm_raw_response.strip().lower()

            # --- CRITICAL: VALIDATION AND MEMORY CHECK ---
            if recommended_product == "none":
                print(f"Agent {self.agent_id}: LLM returned 'None'.")
                return None
            elif recommended_product in current_owned_products:
                print(f"Agent {self.agent_id}: LLM recommended '{recommended_product}', but customer already owns it. Ignoring.")
                return None
            elif recommended_product in past_rec_product_names:
                print(f"Agent {self.agent_id}: LLM recommended '{recommended_product}', but it was already recommended. Ignoring.")
                return None
            elif recommended_product not in self.environment.get_all_products():
                print(f"Agent {self.agent_id}: LLM recommended unknown product '{recommended_product}'. Ignoring.")
                return None
            else:
                # Final eligibility check for LLM suggested product
                product_details = self.environment.get_product_details(recommended_product)
                if product_details:
                    eligible = True
                    # Check eligibility criteria
                    if "min_credit_score" in product_details["eligibility"] and customer_profile["credit_score"] < product_details["eligibility"]["min_credit_score"]:
                        eligible = False
                    if "min_income" in product_details["eligibility"] and customer_profile["income"] < product_details["eligibility"]["min_income"]:
                        eligible = False
                    if "min_age" in product_details["eligibility"] and customer_profile["age"] < product_details["eligibility"]["min_age"]:
                        eligible = False
                    if "max_age" in product_details["eligibility"] and customer_profile["age"] > product_details["eligibility"]["max_age"]:
                        eligible = False

                    if eligible:
                        print(f"Agent {self.agent_id}: LLM recommended '{recommended_product}' (validated and new).")
                        return recommended_product
                    else:
                        print(f"Agent {self.agent_id}: LLM recommended '{recommended_product}' but customer is not eligible. Ignoring.")
                        return None
                else:
                    print(f"Agent {self.agent_id}: Internal error: Product '{recommended_product}' found in all_products but details are missing.")
                    return None

        except Exception as e:
            print(f"Agent {self.agent_id}: Error during LLM reasoning: {e}")
            # This general exception catch should be refined in a real system
            return None

    def act(self, customer_id, recommendation):
        """
        Performs an action based on the reasoning (e.g., send a notification, update a CRM).
        """
        if recommendation:
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S EST") # Using EST as per user preference
            print(f"Agent {self.agent_id}: ACT: Recommending '{recommendation}' to customer {customer_id} at {timestamp}.")
            self.memory.record_recommendation(customer_id, recommendation, timestamp)
            # In a real system, this would trigger an actual integration:
            # - Send a personalized email/push notification
            # - Update CRM with recommendation
            # - Create a task for a human agent
        else:
            print(f"Agent {self.agent_id}: ACT: No action taken for customer {customer_id}.")

    def run(self, customer_id):
        """Simulates the agent's full cycle for a given customer."""
        print(f"\n--- Agent {self.agent_id} Cycle for Customer {customer_id} ---")
        customer_profile = self.perceive(customer_id)
        if customer_profile:
            recommendation = self.reason(customer_profile)
            self.act(customer_id, recommendation)
        print(f"--- End Agent {self.agent_id} Cycle for Customer {customer_id} ---")

# --- Simulation ---
if __name__ == "__main__":
    # --- LLM Client Setup (Corrected Mock) ---

    # Mock the XAI SDK classes as per the user's provided structure
    class MockChatCreateClient:
        def append(self, message):
            pass
        def sample(self):
            class MockSampleResponse:
                # Default content, will be overridden by agent's specific mock_responses
                content = "default_mock_product"
                def __init__(self, content_val=""):
                    self.content = content_val # Allow setting content via constructor
            return MockSampleResponse() # Return an instance with default content

    class MockXAIClient:
        class MockChat:
            def create(self, model, temperature):
                return MockChatCreateClient() # Returns an object that has 'append' and 'sample' methods.

        @property
        def chat(self):
            return self.MockChat()

    # We also need to mock the `user` and `system` message types from `xai_sdk.chat`
    # which are used in `chat.append()`.
    class MockSystemMessage:
        def __init__(self, content): self.content = content
    class MockUserMessage:
        def __init__(self, content): self.content = content

    # Patch the global names expected by the agent's reason method if they are not truly imported
    try:
        from xai_sdk.chat import user, system # Try to import real ones
    except ImportError:
        user = MockUserMessage
        system = MockSystemMessage

    # Initialize the mocked client
    client = MockXAIClient()
    print("XAI SDK Client (mocked) initialized.")
    # --- End LLM Client Setup ---

    # Initialize the environment and memory
    env = BankingEnvironment()
    mem = AgentMemory()

    # Create the Cross-Selling Agent, passing the LLM client
    cross_selling_agent = CrossSellingAgent("NBS_Agent_001", env, mem, client)

    # Run the agent for different customers
    cross_selling_agent.run("customer_123")
    cross_selling_agent.run("customer_456")
    cross_selling_agent.run("customer_789")

    # Demonstrate re-running for a customer (should now show memory effect more clearly)
    print("\n--- Re-running for customer_123 to show memory effect ---")
    cross_selling_agent.run("customer_123")

XAI SDK Client (mocked) initialized.
BankingEnvironment initialized.
BankingEnvironment initialized.
AgentMemory initialized.

CrossSellingAgent 'NBS_Agent_001' initialized with goal: 'Maximize customer value through highly relevant and personalized product recommendations.'

--- Agent NBS_Agent_001 Cycle for Customer customer_123 ---

Agent NBS_Agent_001: Perceiving data for customer customer_123...
Agent NBS_Agent_001: Perceived customer profile: {'age': 35, 'income': 75000, 'current_products': ['checking_account', 'savings_account'], 'recent_activity': ['large_deposit', 'mortgage_inquiry_web'], 'credit_score': 780, 'location': 'urban', 'customer_id': 'customer_123'}
Agent NBS_Agent_001: Reasoning for customer customer_123 using LLM...
Agent NBS_Agent_001: (Simulating LLM call for customer customer_123)
Agent NBS_Agent_001: LLM Raw Response: 'mortgage'
Agent NBS_Agent_001: LLM recommended 'mortgage' (validated and new).
Agent NBS_Agent_001: ACT: Recommending 'mortgage' to customer cu